In [26]:
using Pkg
using CSV 
using DataFrames
using Dates
using CUDA
using Flux
using Statistics
using Random
using ProgressBars
using Gadfly
using ProgressBars
using Cairo

In [3]:
#Pkg.add("ProgressLogging")

In [4]:
filename = "/u/sauves/leucegene-shared/Data/LEUCEGENE/lgn_pronostic_GE_CDS_TPM.csv"
#GE_TRSC_TPM = DataFrame(CSV.File(filename))
@time GE_CDS_TPM = CSV.read(filename, DataFrame)
print()

 15.351611 seconds (41.09 M allocations: 2.115 GiB, 4.87% gc time)


In [5]:
mutable struct Data
    name::String
    data::Array
    factor_1::Array
    factor_2::Array
end 

In [6]:
mutable struct FoldData 
    name::String 
    train::Data
    test::Data
end 

In [11]:
index = GE_CDS_TPM[:,1]
data = GE_CDS_TPM[:,2:end] 
cols = names(data)
# log transforming
data = log10.(Array(data) .+ 1)
# remove least varying genes
ge_var = var(data,dims = 1) 
ge_var_med = median(ge_var)
# high variance only 
hvg = getindex.(findall(ge_var .> ge_var_med),2)[1:Int(floor(end/10))]
data = data[:,hvg]
cols = cols[hvg]
# verify that the operation worked
sum(var(data, dims = 1) .< ge_var_med)
# split into train test
nsamples = length(index)
indices = shuffle(Array{Int}(1:nsamples))
nfolds = 5
foldsize = Int(nsamples / 5)

folds = Array{FoldData}(undef,5)
for i in 1:nfolds
    tst_idx = indices[(i - 1) * foldsize + 1: i * foldsize]
    tr_idx = setdiff(indices, tst_idx)
    test = Data("test", data[tst_idx,:], index[tst_idx], cols)
    train = Data("train", data[tr_idx,:], index[tr_idx], cols)
    fold_data = FoldData("fold_$i", train, test)
    folds[i] = fold_data
end
folds[1].train.factor_1
folds[1].train.data





240×979 Matrix{Float64}:
 0.0316661   0.378646  0.0616036  0.582353  …  1.53271  1.65251  0.641819
 0.607632    0.919941  0.247202   0.825677     1.36331  1.68384  0.747082
 0.0542269   0.858341  0.168166   0.353205     1.6256   2.01853  0.638438
 0.117943    0.909923  0.458532   0.512683     1.59041  1.44073  0.601077
 0.1027      0.92438   0.0417026  0.927698     1.67489  1.70492  0.741999
 0.632719    1.1266    0.326749   0.743149  …  2.10699  1.72211  0.669462
 0.0639534   0.988574  0.344356   0.509409     1.67006  1.55408  0.53655
 0.171966    0.878851  0.411984   0.124825     1.61074  1.58294  0.419827
 0.436952    0.956903  0.0219285  0.391883     1.12661  1.48788  0.404533
 0.0573734   0.591869  0.0818238  0.454019     1.22687  1.10439  0.251101
 0.0553016   0.88431   0.248367   0.115296  …  1.63206  1.31407  0.416914
 0.225357    0.899478  0.122377   0.213399     1.86575  1.5498   0.676836
 0.972166    1.27069   0.422964   0.571621     1.79595  1.87913  0.586622
 ⋮            

In [13]:
function prep_data(data::Data; device = gpu)
    ## data preprocessing
    ### remove index columns, log transform
    n = length(data.factor_1)
    m = length(data.factor_2)
    values = Array{Float32,2}(undef, (1, n * m))
    #print(size(values))
    factor_1_index = Array{Int32,1}(undef, n * m)
    factor_2_index = Array{Int32,1}(undef, n * m)
    # d3_index = Array{Int32,1}(undef, n * m)
    for i in 1:n
        for j in 1:m
            index = (i - 1) * m + j 
            values[1, index] = data.data[i, j]
            factor_1_index[index] = i # Int
            factor_2_index[index] = j # Int 
            # d3_index[index] = data.d3_index[i] # Int 
        end
    end
    return (device(factor_1_index), device(factor_2_index)), device(values)
end 

prep_data (generic function with 1 method)

In [14]:
function custom_train!(loss, ps, data, opt, loss_data, epoch)
    loss_array = Array{Float32,1}(undef, length(data))

    for (i, (x, y)) in enumerate(data)
        loss_array[i] = loss(x, y)
        
        gs = gradient(ps) do
            loss(x, y)
        end
        Flux.update!(opt, ps, gs)
    end

    loss_data[epoch] = mean(loss_array)
end


custom_train! (generic function with 1 method)

In [15]:
function nn(factor_1_layer, factor_2_layer, (a, b, c))
    return Chain(
        Flux.Parallel(vcat, factor_1_layer, factor_2_layer),
        Dense(a, b, relu),
        Dense(b, c, relu),
        Dense(c, 1, identity)
    )
end

nn (generic function with 1 method)

In [22]:
function get_obtained_expected(X, Y, network, device)
    nb = 10000
    index = rand(1:length(Y), nb)

    obtained = cpu(Y)[index]

    X_cpu = cpu(X)
    factor_1 = X_cpu[1][index]
    factor_2 = X_cpu[2][index]
    
    expected = cpu(network((device(factor_1), device(factor_2))))

    return obtained, expected[1,:]
end

get_obtained_expected (generic function with 1 method)

In [23]:

function plot_accuracy(folder, epoch, obtained, expected)
    corP = Statistics.cor(obtained, expected)
    draw(
        PNG("$(folder)/accuracy_at_epoch_$epoch.png"), 
        plot(
            x=expected, y=obtained, Guide.title("Accuracy at epoch $epoch (r = $corP)"), 
            Guide.xlabel("Expected"), Guide.ylabel("Obtained"), Geom.abline, 
            Geom.hexbin(xbincount=100, ybincount=100)
        )
    )
end

plot_accuracy (generic function with 1 method)

In [24]:
function train_embeddings(dl, X_, Y_, (dims1, dims2), folder, data, epochs = 200; device = gpu)
    #prepare outpath
    mkdir("$(folder)")
    # construct embeddings & the neural net
    d1_layer = device(Flux.Embedding(length(data.factor_1), dims1))
    d2_layer = device(Flux.Embedding(length(data.factor_2), dims2))
    
    a = dims1 + dims2 
    b = 50
    c = 10 

    network = device(nn(d1_layer, d2_layer, (a, b, c)))

    loss(x, y) = Flux.Losses.mse(network(x), y)
    ps = Flux.params(network)
    tr = 0.001
    opt = Flux.ADAM(tr)

    neural_network = ("- Layer 1 : Embeddings\n" * 
        "- Layer 2 : Dense($a, $b, relu)\n" *
        "- Layer 3 : Dense($b, $c, relu)\n" *
        "- Layer 4 : Dense($c, 1, identity)"
    )
    println(neural_network)
    loss_data = Array{Float32,1}(undef, epochs)
    # cycle through epochs
    for e in ProgressBar(1:epochs)
        custom_train!(loss, ps, dl, opt, loss_data, e)
        obtained, expected = get_obtained_expected(X_, Y_, network, device)
        #println(iter, "Prepare intermediate accuracy plot at epoch $(i)...")
        plot_accuracy(folder, e, obtained, expected)
        
    
    end
    println("final loss: $(loss_data[end])")

end

train_embeddings (generic function with 2 methods)

In [28]:
outpath = "output"
mb_size = 2 ^ 14
fold_data = folds[1]

X_tr, Y_tr = prep_data(fold_data.train)
train_dl =  Flux.Data.DataLoader((X_tr, Y_tr), batchsize = mb_size)
X_tst, Y_tst = prep_data(fold_data.test)
test_dl = Flux.Data.DataLoader((X_tst, Y_tst), batchsize = mb_size)

train_embeddings(train_dl, X_tr, Y_tr, (2,2), "$(outpath)/embeddings_$(now())", fold_data.train, 100)


- Layer 1 : Embeddings
- Layer 2 : Dense(4, 50, relu)
- Layer 3 : Dense(50, 10, relu)
- Layer 4 : Dense(10, 1, identity)


0.0%┣                                              ┫ 0/100 [00:00<00:-5, -0s/it]
1.0%┣▍                                         ┫ 1/100 [00:00<Inf:Inf, InfGs/it]
2.0%┣█                                              ┫ 2/100 [00:00<00:31, 3it/s]
3.0%┣█▍                                             ┫ 3/100 [00:00<00:19, 5it/s]
4.0%┣█▉                                             ┫ 4/100 [00:00<00:15, 6it/s]
5.0%┣██▍                                            ┫ 5/100 [00:01<00:13, 7it/s]
6.0%┣██▉                                            ┫ 6/100 [00:01<00:12, 8it/s]
7.0%┣███▎                                           ┫ 7/100 [00:01<00:11, 9it/s]
8.0%┣███▊                                           ┫ 8/100 [00:01<00:10, 9it/s]
9.0%┣████▎                                          ┫ 9/100 [00:01<00:10, 9it/s]
10.0%┣████▌                                        ┫ 10/100 [00:01<00:10, 9it/s]
11.0%┣█████                                        ┫ 11/100 [00:01<00:09, 9it/s]
12.0%┣█████▎                

final loss: 0.08866258


100.0%┣██████████████████████████████████████████┫ 100/100 [00:08<00:00, 12it/s]
100.0%┣██████████████████████████████████████████┫ 100/100 [00:08<00:00, 12it/s]
